In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [ ]:
CSV_files ={"customers": r"../data/olist_customers_dataset.csv",
"geolocation_dataset":r"../data/olist_geolocation_dataset.csv",
"order_items":r"../data/olist_order_items_dataset.csv",
"order_payments":r"../data/olist_order_payments_dataset.csv",
"order_review":r"../data/olist_order_reviews_dataset.csv",
"order_dataset":r"../data/olist_orders_dataset.csv",
"list_product":r"../data/olist_products_dataset.csv",
"list_seller":r"../data/olist_sellers_dataset.csv",
"product_category":r"../data/product_category_name_translation.csv"}

In [24]:
dataframes = {name: pd.read_csv(path) for name, path in CSV_files.items()}

In [25]:
df_orders_items = dataframes["order_items"]
df_order_payments = dataframes["order_payments"]
df_order_review= dataframes["order_review"]
df_order_dataset = dataframes["order_dataset"]
df_list_product = dataframes["list_product"]
df_list_seller = dataframes["list_seller"]
df_product_category = dataframes["product_category"]
df_geolocation_dataset = dataframes["geolocation_dataset"]

In [26]:
df_list_product.info()#individuati 610 valori nulli sul df list_:product

<class 'pandas.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  str    
 1   product_category_name       32341 non-null  str    
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), str(2)
memory usage: 2.3 MB


In [27]:
df_list_product

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [28]:
product_id_nan=df_list_product[df_list_product['product_category_name'].isna()]
product_id_to_delete=product_id_nan['product_id']
product_id_to_delete=product_id_to_delete.to_numpy()
#creato un array con gli id prodototto che non hanno categorie per usarlo come criteri di esclusione negli altri df

In [29]:
df_list_product=df_list_product.dropna()
df_list_product.info()#pulito il df  da i valori nulli probabilemnte ho perso 1 id product 32340 invece di  32341

<class 'pandas.DataFrame'>
Index: 32340 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32340 non-null  str    
 1   product_category_name       32340 non-null  str    
 2   product_name_lenght         32340 non-null  float64
 3   product_description_lenght  32340 non-null  float64
 4   product_photos_qty          32340 non-null  float64
 5   product_weight_g            32340 non-null  float64
 6   product_length_cm           32340 non-null  float64
 7   product_height_cm           32340 non-null  float64
 8   product_width_cm            32340 non-null  float64
dtypes: float64(7), str(2)
memory usage: 2.5 MB


In [30]:
df_orders_items_clean=df_orders_items[~df_orders_items['product_id'].isin(product_id_to_delete)]
df_orders_items_clean.info()#creato il dataframe df_orders ripulito dai prodotti senza categoria

<class 'pandas.DataFrame'>
Index: 111047 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             111047 non-null  str    
 1   order_item_id        111047 non-null  int64  
 2   product_id           111047 non-null  str    
 3   seller_id            111047 non-null  str    
 4   shipping_limit_date  111047 non-null  str    
 5   price                111047 non-null  float64
 6   freight_value        111047 non-null  float64
dtypes: float64(2), int64(1), str(4)
memory usage: 6.8 MB
